In [83]:
import urllib
import os
import collections
import numpy as np
from tqdm import tqdm
import torch
from torch import nn
import torchtext
from torch.utils.data import Dataset, DataLoader, random_split

In [84]:
os.makedirs('data', exist_ok=True)
dataset_train, dataset_test = torchtext.datasets.AG_NEWS(root='./data')
tokenizer = torchtext.data.utils.get_tokenizer('basic_english', language="en")
dataset_train = list(dataset_train)
dataset_test = list(dataset_test)

In [85]:
tokenized_data = []
label_data = []
counter = collections.Counter()

print('Start Tokenizing...')
for label, line in tqdm(dataset_train):
    tokens = tokenizer(line)
    tokenized_data.append(tokens)
    label_data.append(label-1)

print('Making Vocab...')
for line in tqdm(tokenized_data):
    counter.update(line)

vocab = torchtext.vocab.vocab(counter, min_freq=1, specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"])

Start Tokenizing...


100%|██████████| 120000/120000 [00:04<00:00, 27876.49it/s]


Making Vocab...


100%|██████████| 120000/120000 [00:01<00:00, 100627.96it/s]


In [86]:
class AGNewsDataset(Dataset):
    def __init__(self, tokenized_data, label_data, max_seq = 256):
        self.x = []
        self.y = []
        self.classes = ['World', 'Sports', 'Business', 'Sci/Tech']
        
        for tokens in tqdm(tokenized_data):
            token_ids = [vocab[token] for token in tokens]
            self.x.append(token_ids)
        self.y = label_data
        
    def __len__(self):
        return len(self.x)
        
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx]) , torch.tensor(self.y[idx])

In [87]:
news_dataset = AGNewsDataset(tokenized_data, label_data, max_seq=256)

100%|██████████| 120000/120000 [00:04<00:00, 28106.76it/s]


In [88]:
train_ratio = 0.8
train_size = int(train_ratio*len(news_dataset))
valid_size = len(news_dataset) - train_size
train_dataset, valid_dataset = random_split(news_dataset, [train_size, valid_size])

In [89]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.rnn = nn.RNN(embedding_size, hidden_size, batch_first=True)
        self.out =  nn.Linear(hidden_size, output_size)

    def forward(self, input):
        input = self.embedding(input)
        #input: [batch_size, sentence_len, emb_size]
        output, _ = self.rnn(input)
        #output: [batch_size, sentence_len, hidden_size]
        output = self.out(output)
        #output: [batch_size, sentence_len, output_size]
        output = output.transpose(1,0)
        #output: [sentence_len, batch_size, output_size]
        output = output[-1]
        #output: [batch_size, output_size]

        return output



In [90]:
model = RNN(vocab_size=len(vocab), embedding_size=64, hidden_size=32, output_size=len(news_dataset.classes))

In [91]:
batch_size=256
learning_rate = 0.001
num_epochs = 10
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [92]:
def pad_collate_fn(batch):
    collate_x = []
    collate_y = []
    for data, label in batch:
        collate_x.append(data)
        collate_y.append(label)
    collate_x = nn.utils.rnn.pad_sequence(collate_x, padding_value=vocab['<pad>'], batch_first=True)
    collate_y = torch.stack(collate_y)
    return (collate_x, collate_y)



In [93]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn= pad_collate_fn)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn= pad_collate_fn)

In [94]:
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [95]:
def accuracy(output, label):
    pred = torch.argmax(output, dim=-1)
    acc = torch.sum(pred==label)
    return acc
    

In [96]:
def train(dataloader, epoch):
    model.train()
    epoch_loss = 0
    epoch_acc = 0
    for data,label in tqdm(dataloader, desc=f"Epoch {epoch}"):
        data, label = data.to(device), label.to(device)
        output = model(data)
        optimizer.zero_grad()
        loss = criterion(output, label)
        acc = accuracy(output, label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    print(f"Train Epoch: {epoch}, Loss: {epoch_loss/len(dataloader)}, Acc: {epoch_acc/len(dataloader)}")


In [97]:
def evaluate(dataloader, epoch):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    with torch.no_grad():
        for data,label in tqdm(dataloader, desc=f"Epoch {epoch}"):
            data, label = data.to(device), label.to(device)
            output = model(data)
            loss = criterion(output, label)
            acc = accuracy(output, label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()

    print(f"Evaluate Epoch: {epoch}, Loss: {epoch_loss/len(dataloader)}, Acc: {epoch_acc/len(dataloader)}")

In [98]:
total_acc = 0
for epoch in range(num_epochs):
    train(train_dataloader, epoch)
    evaluate(valid_dataloader, epoch)
    print('-'*50)



Epoch 0: 100%|██████████| 375/375 [00:05<00:00, 66.87it/s]


Train Epoch: 0, Loss: 1.3867614844640097, Acc: 64.36266666666667


Epoch 0: 100%|██████████| 94/94 [00:00<00:00, 97.94it/s] 


Evaluate Epoch: 0, Loss: 1.386565424026327, Acc: 63.01063829787234
--------------------------------------------------


Epoch 1: 100%|██████████| 375/375 [00:05<00:00, 64.72it/s]


...

Evaluate Epoch: 8, Loss: 1.3864884465298755, Acc: 63.04255319148936
--------------------------------------------------


Epoch 9: 100%|██████████| 375/375 [00:05<00:00, 68.45it/s]


Train Epoch: 9, Loss: 1.3863102947870891, Acc: 63.928


Epoch 9: 100%|██████████| 94/94 [00:00<00:00, 102.79it/s]

Evaluate Epoch: 9, Loss: 1.3864680997868801, Acc: 63.06382978723404
--------------------------------------------------
